<a href="https://colab.research.google.com/github/ja390/Parallel-Computing-Assignment3/blob/main/Serial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt-get update -qq

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
!sudo apt-get install -y openmpi-bin libopenmpi-dev > /dev/null

In [3]:
%%writefile serial_kmeans.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>

#define MAX_ITER 100
#define K 3   // Number of clusters

typedef struct {
    double x, y;
    int cluster;
} Point;

typedef struct {
    double x, y;
} Centroid;

// Euclidean distance
double distance(Point p, Centroid c) {
    return sqrt((p.x - c.x)*(p.x - c.x) + (p.y - c.y)*(p.y - c.y));
}

int main() {
    int n = 9; // number of points
    Point points[9] = {
        {1,2}, {2,1}, {3,3},
        {8,8}, {9,8}, {8,9},
        {4,5}, {5,6}, {6,5}
    };

    // Initial centroids
    Centroid centroids[K] = {{2,2}, {8,8}, {5,5}};

    printf("---- Serial K-Means Results ----\n");

    for (int iter = 0; iter < MAX_ITER; iter++) {

        // Assign points to nearest centroid
        for (int i = 0; i < n; i++) {
            double minDist = 1e9;
            int bestCluster = 0;

            for (int j = 0; j < K; j++) {
                double dist = distance(points[i], centroids[j]);
                if (dist < minDist) {
                    minDist = dist;
                    bestCluster = j;
                }
            }

            points[i].cluster = bestCluster;
        }

        // Recompute centroids
        double sumX[K] = {0}, sumY[K] = {0};
        int count[K] = {0};

        for (int i = 0; i < n; i++) {
            int c = points[i].cluster;
            sumX[c] += points[i].x;
            sumY[c] += points[i].y;
            count[c]++;
        }

        for (int j = 0; j < K; j++) {
            if (count[j] > 0) {
                centroids[j].x = sumX[j] / count[j];
                centroids[j].y = sumY[j] / count[j];
            }
        }
    }

    printf("\nFinal Cluster Assignments:\n");
    for (int i = 0; i < n; i++) {
        printf("Point (%.2f, %.2f) → Cluster %d\n",
               points[i].x, points[i].y, points[i].cluster);
    }

    printf("\nFinal Centroids:\n");
    for (int j = 0; j < K; j++) {
        printf("Centroid %d: (%.2f, %.2f)\n", j, centroids[j].x, centroids[j].y);
    }

    return 0;
}


Writing serial_kmeans.c


In [4]:
!gcc serial_kmeans.c -o serial_kmeans -lm


In [5]:
!./serial_kmeans


---- Serial K-Means Results ----

Final Cluster Assignments:
Point (1.00, 2.00) → Cluster 0
Point (2.00, 1.00) → Cluster 0
Point (3.00, 3.00) → Cluster 0
Point (8.00, 8.00) → Cluster 1
Point (9.00, 8.00) → Cluster 1
Point (8.00, 9.00) → Cluster 1
Point (4.00, 5.00) → Cluster 2
Point (5.00, 6.00) → Cluster 2
Point (6.00, 5.00) → Cluster 2

Final Centroids:
Centroid 0: (2.00, 2.00)
Centroid 1: (8.33, 8.33)
Centroid 2: (5.00, 5.33)
